In [188]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [189]:
df = pd.read_excel('AtlasBrasil_Consulta.xls')

In [190]:
df.head()

,Lugar,Espacialidade,COD IBGE,IDHM Renda (2010),IDHM Longevidade (2010),IDHM Educação (2010),Taxa de analfabetismo - 11 a 14 anos (2010),Taxa de analfabetismo - 15 a 17 anos (2010),Taxa de analfabetismo - 18 a 24 anos (2010),Taxa de analfabetismo - 25 a 29 anos (2010),Taxa de analfabetismo - 25 anos ou mais (2010),Taxa de analfabetismo - 15 anos ou mais (2010),Taxa de analfabetismo - 18 anos ou mais (2010),Renda per capita (2010)
0,Brasil,País,--,0.739,0.816,0.637,3.24,2.20,2.61,3.96,11.82,9.61,10.19,793.87
1,Aeroviários / Esplanada Anicuns / São José / A...,UDH,--,0.766,0.878,0.736,0.74,0.25,0.28,0.42,3.10,2.44,2.56,942.98
2,Água Branca / Riviera / Sonho Verde / Califórn...,UDH,--,0.786,0.908,0.775,0.78,0.64,0.60,0.59,3.46,2.81,2.95,1063.54
3,"Aldeia do Vale / Monte Verde (Goiânia , RM Goi...",UDH,--,1.000,0.940,0.920,0.00,0.00,0.88,0.00,0.33,0.38,0.40,6361.16
4,Alphaville Flamboyant / Jardins Munique / Port...,UDH,--,1.000,0.940,0.920,0.27,0.70,0.32,0.26,0.45,0.44,0.43,6361.16


In [191]:
df.columns



Index(['Lugar', 'Espacialidade', 'COD IBGE', 'IDHM Renda (2010)',
       'IDHM Longevidade (2010)', 'IDHM Educação (2010)',
       'Taxa de analfabetismo - 11 a 14 anos (2010)',
       'Taxa de analfabetismo - 15 a 17 anos (2010)',
       'Taxa de analfabetismo - 18 a 24 anos (2010)',
       'Taxa de analfabetismo - 25 a 29 anos (2010)',
       'Taxa de analfabetismo - 25 anos ou mais (2010)',
       'Taxa de analfabetismo - 15 anos ou mais (2010)',
       'Taxa de analfabetismo - 18 anos ou mais (2010)',
       'Renda per capita (2010)'],
      dtype='object')

In [192]:

#df.loc[df['Lugar'].str.contains('(Goiânia , RM Goiânia)'),'REGIAO_METROPOLITANA'] = True
#df.loc[~df['Lugar'].str.contains('(Goiânia , RM Goiânia)'),'REGIAO_METROPOLITANA'] = False

In [193]:
dfFinal = pd.DataFrame(columns=df.columns)
dfFinal

,Lugar,Espacialidade,COD IBGE,IDHM Renda (2010),IDHM Longevidade (2010),IDHM Educação (2010),Taxa de analfabetismo - 11 a 14 anos (2010),Taxa de analfabetismo - 15 a 17 anos (2010),Taxa de analfabetismo - 18 a 24 anos (2010),Taxa de analfabetismo - 25 a 29 anos (2010),Taxa de analfabetismo - 25 anos ou mais (2010),Taxa de analfabetismo - 15 anos ou mais (2010),Taxa de analfabetismo - 18 anos ou mais (2010),Renda per capita (2010)


In [194]:
# este codigo serve para pegar mais de um bairro que tem mais de um complemento na mesma linha     
dfFinal = pd.DataFrame(columns=df.columns)
dfFinal['Complemento'] = np.NaN
dfFinal['Id_linha_original'] = 0
for i, row in df.iterrows():
     row['Id_linha_original'] = i+2 # criei este numero para ter um controle de qual linha originou os novos registros
                                    # esse + 2 é por conta da linha de cabeçalho no .xls e a contagem iniciar do 0
    # retirando (Goiania, RM Goiania) 
     ii = row['Lugar'].find('(Goiânia , RM Goiânia)')
     lugar  = row['Lugar']
     if ii != -1:
         lugar = lugar[0:ii]
            
     # existem vários padroes no arquivo, mas em grande parte está separado por barras /
     # alguns possuem um ou mais dois pontos':'
     # então a estratégia é pegar o que vem antes do ':' e assumir como bairro, e o seguinte como complemento
     # para os complementos que possuem /, serão divididos em novas linhas, uma linha pra cada, invés de deixar agrupado
     # quando houver mais que um ':' , pegar o texto entre a barra anterior e os dois pontos como bairro, criando nova linha
    
    
     #criando o array de bairros 
     bairros= []
     if lugar.count(":")>1 :
         import re
         
         indexponto = [m.start() for m in re.finditer(':', lugar)] # esta linha pega o index de todas as ocorrencias de ':'
        
         
        
         i = 0 #era pra ser um for, mas o python n faz o for classico
         while i < len(indexponto):
            bairros =[]
            #pega o nome do bairro
            nomeBairro = lugar[0:indexponto[i]] # por padrao pega do inicio ao ':'
            if i !=0 and nomeBairro.rfind("/")>0:  # se não for a primeira iteracao e houver barra
                nomeBairro = lugar[nomeBairro.rfind("/")+1:indexponto[i]] # o nome do bairro sera da barra até os pontos
           
            #pega o complemento
            complementoBruto = lugar[indexponto[i]+1:len(lugar)]# este +1 é pra pular os dois pontos
            if i+1<len(indexponto)  : 
                complementoBruto = lugar[indexponto[i]+1:indexponto[i+1]]
                complementoBruto = complementoBruto[0:complementoBruto.rfind("/")]
                
            
            for comp in complementoBruto.split("/") : 
                bairros.append(nomeBairro +" ( " + comp +" )") # formata para o a forma padrao de processamento
                
            i = i+1    
                
     elif lugar.count(":")==1 : # processo similar ao anterior
         complemento = lugar[lugar.find(":")+1:len(lugar)]
         lugar  = lugar[0:lugar.find(":")]
         for comp in complemento.split("/") :
             bairros.append(lugar +" ("+ comp+")")   
         
     else:
         bairros = lugar.split("/") # processo padrão, na maioria das aglomerações, os bairros sao separadas por barras
         
        #reescrive tudo em um novo DataFrame, processando todos da mesma forma
     for bairro in bairros:        
           complemento = np.NaN
           ii = bairro.find('(')
           if ii != -1 :
                bairro = bairro[:-2]
                    
           if ii != -1 :
                complemento = bairro[ii+1:len(bairro)].strip()
                bairro = bairro[0:ii]
                
           
           rowTeste = row        
           rowTeste['Complemento'] = complemento
           rowTeste['Lugar'] = bairro.strip()
           dfFinal.loc[dfFinal.size] = rowTeste

In [195]:
for i, row in dfFinal.iterrows():
     if row['Lugar'].find('Unidade Territorial de Planejamento')==0 :
        dfFinal.set_value(index=i,col='Lugar',value=row['Lugar'][36:])
       

In [196]:
dfFinal.to_excel('AtlasBairrosTratadosSemCocheia.xls')